In [ ]:
import pandas as pd
import pymysql

In [ ]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Passw0rd123',
    'database': 'retail_sales_dw'
}

In [ ]:
customer_vip_data = pd.read_csv('./customer_vip_info.csv')
customer_vip_data.iloc[:, 2:] = customer_vip_data.iloc[:, 2:].astype(bool)

In [ ]:
conn = pymysql.connect(**db_config)
customer_dim = pd.read_sql('SELECT customer_id, first_name, last_name, phone, credit_limit FROM customer_dim', conn)

merged_df = pd.merge(customer_dim, customer_vip_data, on=['first_name', 'last_name'], how='left')

C:\Users\ds2002-student\AppData\Local\Temp\2\ipykernel_8136\2674082246.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_dim = pd.read_sql('SELECT customer_id, first_name, last_name, phone, credit_limit FROM customer_dim', conn)


In [ ]:
merged_df['is_vip'] = merged_df['is_vip'].fillna(False)
conn = pymysql.connect(**db_config)
cursor = conn.cursor()

# cursor.execute("ALTER TABLE customer_dim DROP COLUMN is_vip")
cursor.execute("ALTER TABLE customer_dim ADD COLUMN is_vip BOOL")

0

In [ ]:
merged_df.head(10)

,customer_id,first_name,last_name,phone,credit_limit,is_vip
0,103,Carine,Schmitt,40.32.2555,21000.0,False
1,112,Jean,King,7025551838,71800.0,True
2,114,Peter,Ferguson,03 9520 4555,117300.0,True
3,119,Janine,Labrune,40.67.8555,118200.0,True
4,121,Jonas,Bergulfsen,07-98 9555,81700.0,False
5,124,Susan,Nelson,4155551450,210500.0,False
6,125,Zbyszek,Piestrzeniewicz,(26) 642-7555,0.0,True
7,128,Roland,Keitel,+49 69 66 90 2555,59700.0,True
8,129,Julie,Murphy,6505555787,64600.0,True
9,131,Kwai,Lee,2125557818,114900.0,False


In [ ]:
for index, row in merged_df.iterrows():
    update_query = """
        UPDATE customer_dim
        SET is_vip = %s
        WHERE customer_id = %s
    """
    values = (row['is_vip'], row['customer_id'])
    cursor.execute(update_query, values)

conn.commit()
conn.close()